In [2]:
import tensorflow as tf
import yaml
import sys
# point to models/research directory of tensorflow
sys.path.append("../models/research") 
from object_detection.utils import dataset_util

In [3]:
LABEL_DICT = {'Green': 1,
             'GreenLeft': 2,
             'GreenRight': 3,
             'GreenStraight': 4,
             'GreenStraightLeft': 5,
             'GreenStraightRight': 6,
             'Red': 7,
             'RedLeft': 8,
             'RedRight': 9,
             'RedStraight': 10,
             'RedStraightLeft': 11,
             'Yellow': 12,
             'off': 13
             }

def create_tf_example(example):
    # TODO(user): Populate the following variables from your example.
    height = 720 # Image height
    width = 1280 # Image width
    filename = example['path'] # Filename of the image. Empty if image is not from file
    filename = filename.encode()
    with tf.gfile.GFile(example['path'], 'rb') as fid:
        encoded_image = fid.read()
    image_format = 'png'.encode() # b'jpeg' or b'png'

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
             # (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List o'f integer class id of bounding box (1 per box)

    for box in example['boxes']:
        xmins.append(float(box['x_min']/width))
        xmaxs.append(float(box['x_max']/width))
        ymins.append(float(box['y_min']/height))
        ymaxs.append(float(box['y_max']/height))
        classes_text.append(box['label'].encode())
        classes.append(int(LABEL_DICT[box['label']]))
        
    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_image),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [4]:
writer = tf.python_io.TFRecordWriter("./bosch.record")

In [14]:
examples = yaml.load(open("../dataset_train_rgb/train.yaml", "rb").read())

In [15]:
from collections import Counter
import os
PATH_TO_BOSCH_DATA_SET = "../dataset_train_rgb"
labels = []
for example in examples:
    example['path'] = os.path.join(PATH_TO_BOSCH_DATA_SET, *example['path'].split("/")[1:])

In [16]:
example['path']

'../dataset_train_rgb/rgb/train/2015-10-05-16-02-30_bag/720932.png'

In [17]:
for example in examples:
    tf_example = create_tf_example(example)
    writer.write(tf_example.SerializeToString())
writer.close()